<a href="https://colab.research.google.com/github/revanthalapati03/Whatatripv1/blob/main/700759015_ICP6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Revanth Alapati

700759015

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('/content/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 23s - 80ms/step - accuracy: 0.6444 - loss: 0.8291
144/144 - 2s - 14ms/step - accuracy: 0.6693 - loss: 0.7587
0.7587000131607056
0.669287919998169
['loss', 'compile_metrics']


In [ ]:
model.save('SentimentAnalysis.h5')

In [ ]:
import tweepy
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

In [ ]:
# Load the saved model
model = load_model('/content/SentimentAnalysis.h5')

# Define a function for preprocessing text
def preprocess_data(text):
  text = text.lower()
  text = re.sub('[^a-zA-z0-9\s]', '', text)
  return text

new_data = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing"
# Preprocess the new text data
new_data = preprocess_data(new_data)

# Tokenize and pad the new text data
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts([new_data])
X_new = tokenizer.texts_to_sequences([new_data])
X_new = pad_sequences(X_new, maxlen=model.input_shape[1])

# Make predictions
predictions = model.predict(X_new)

# Determine the sentiment based on the prediction
sentiments = ['Negative', 'Neutral', 'Positive']
predicted_sentiment = sentiments[predictions.argmax()]

# Print the result
print("Predicted Sentiment: " + predicted_sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Predicted Sentiment: Negative


Apply GridSearchCV on the source code

In [ ]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [ ]:

import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier

# Assuming the data loading and preprocessing steps are the same
model = load_model('/content/SentimentAnalysis.h5')
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
# Assuming tokenizer fitting and text preprocessing is done here

def createmodel(optimizer='adam'):
    model1 = Sequential()
    model1.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model1.add(SpatialDropout1D(0.2))
    model1.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model1.add(Dense(3, activation='softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the KerasClassifier with the build_fn as our model creation function
model = KerasClassifier(model, verbose=2)

# Define hyperparameters to tune
param_grid = {
    'batch_size': [32, 64],
    'epochs': [1, 2],
    'optimizer': ['adam', 'rmsprop']
}

In [ ]:

# Initialize GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)


In [ ]:
# Fit GridSearchCV
grid_result = grid.fit(X_train, Y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


194/194 - 22s - 114ms/step - accuracy: 0.7104 - loss: 0.6909
97/97 - 1s - 15ms/step
194/194 - 11s - 59ms/step - accuracy: 0.7123 - loss: 0.6886
97/97 - 1s - 15ms/step
194/194 - 11s - 58ms/step - accuracy: 0.7063 - loss: 0.6792
97/97 - 1s - 15ms/step
194/194 - 11s - 59ms/step - accuracy: 0.7104 - loss: 0.6883
97/97 - 2s - 20ms/step
194/194 - 11s - 58ms/step - accuracy: 0.7062 - loss: 0.6817
97/97 - 2s - 16ms/step
194/194 - 11s - 55ms/step - accuracy: 0.7103 - loss: 0.6759
97/97 - 3s - 31ms/step
Epoch 1/2
194/194 - 10s - 52ms/step - accuracy: 0.7103 - loss: 0.6913
Epoch 2/2
194/194 - 10s - 54ms/step - accuracy: 0.7393 - loss: 0.6235
97/97 - 1s - 15ms/step
Epoch 1/2
194/194 - 11s - 56ms/step - accuracy: 0.7119 - loss: 0.6802
Epoch 2/2
194/194 - 10s - 50ms/step - accuracy: 0.7411 - loss: 0.6163
97/97 - 1s - 15ms/step
Epoch 1/2
194/194 - 11s - 58ms/step - accuracy: 0.7119 - loss: 0.6725
Epoch 2/2
194/194 - 12s - 61ms/step - accuracy: 0.7489 - loss: 0.5998
97/97 - 2s - 21ms/step
Epoch 1/2
19